# Generate BM 25 for the different datasets
Covid, NFCorpus, Touche, DBPedia, SciFact, MS Marco, Novel eval.

https://github.com/beir-cellar/beir?tab=readme-ov-file

https://github.com/sunnweiwei/RankGPT/tree/main/NovelEval

In [1]:
# download data
# Build up the bm25 searching engine of the pyserini
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
import json
import openai
import re
import pandas as pd
# from load_selfknowledge import simple_search_with_context_bert
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm 

from pyserini.search import LuceneSearcher, get_topics, get_qrels


/home/yuj49/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


dataset_list = {
    # "nfcorpus":{
    #     "train_query":"../data/nfcorpus/train.nontopic-titles.queries",
    #     "train_qrel":"../data/nfcorpus/train.2-1-0.qrel",
    #     "knowledge_base":'beir-v1.0.0-nfcorpus.flat'},
    # "covid": {
    #     "train_query":"../data/trec_covid/queries.jsonl",
    #     "train_qrel":"../data/trec_covid/qrels/test.tsv",
    #     "knowledge_base": "beir-v1.0.0-trec-covid.flat"},
    # "touche":{
    #     "train_query":"../data/touche/queries.jsonl",
    #     "train_qrel":"../data/touche/qrels/test.tsv",
    #     "knowledge_base": "beir-v1.0.0-webis-touche2020.flat"},
    # "dbpedia":{
    #     "train_query":"../data/dbpedia-entity/queries.jsonl",
    #     "train_qrel":"../data/dbpedia-entity/qrels/test.tsv",
    #     "test_qrel":"../data/dbpedia-entity/qrels/dev.tsv",
    #     "knowledge_base": "beir-v1.0.0-dbpedia-entity.flat"},
    # "scifact":{
    #     "train_query":"../data/scifact/queries.jsonl",
    #     "train_qrel":"../data/scifact/qrels/train.tsv",
    #     "test_qrel": "../data/scifact/qrels/test.tsv",
    #     "knowledge_base":"beir-v1.0.0-scifact.flat"},
    "msmarco":{
        "train_query":"../data/msmarco/queries.jsonl",
        "train_qrel":"../data/msmarco/qrels/train.tsv",
        "test_qrel":"../data/msmarco/qrels/test.tsv",
        "dev_qrel":"../data/msmarco/qrels/dev.tsv",
        "knowledge_base": "msmarco-v1-passage"},
    # "noval":{
    #     "train_query":"../data/Noveleval/queries.tsv",
    #     "train_qrel":"../data/Noveleval/qrel.txt",
    #     "corpus": "../data/Noveleval/corpus.tsv"},      
}


def retrieve_documents(query, knowledge_base='beir-v1.0.0-nfcorpus.flat', top_n=20):
    # initialize the retrieval
    # problem here is the knowledge is too long
    searcher = LuceneSearcher.from_prebuilt_index(knowledge_base)
    index_reader = IndexReader.from_prebuilt_index(knowledge_base)
    hits = searcher.search(query, k=top_n)
    documents = []
    if knowledge_base == "msmarco-v1-passage":
        for i, hit in enumerate(hits):
            docid = hit.docid
            score = hit.score
            raw_document = searcher.doc(docid).raw()
            raw_document = json.loads(raw_document)
            raw_document = raw_document["contents"]
            documents.append({"docid": docid, "content": raw_document})
           
    else:
        for i, hit in enumerate(hits):
            docid = hit.docid
            score = hit.score
            raw_document = searcher.doc(docid).raw()
            raw_document = json.loads(raw_document)["text"]
            documents.append({"docid": docid, "content": raw_document})
            
    return documents

def write_query_qrel(query_file, qrel_file, knowledge_base="msmarco-v1-passage", data_name="msmarco", data_type="test", train_limit=100):    
    query_used = []
    # qrel file suppose to be the tsv file 
    with open(qrel_file,"r") as f:
        lines = f.readlines()
        qrel_res = []
        if data_type == "train" and train_limit:
            lines = lines[:train_limit]
        if knowledge_base in["msmarco-v1-passage","beir-v1.0.0-scifact.flat","beir-v1.0.0-webis-touche2020.flat","beir-v1.0.0-trec-covid.flat", "beir-v1.0.0-dbpedia-entity.flat"]:
            for line in lines:
                item = {}
                query_id = line.split("\t")[0]
                doc_id = line.split("\t")[1]
                score = line.split("\t")[-1].strip()
                item["query_id"] = query_id
                query_used.append(query_id)
                item["doc_id"] = doc_id
                item["score"] = score
                qrel_res.append(item)
        else:
            for line in lines:
                item = {}
                query_id = line.split("\t")[0]
                doc_id = line.split("\t")[2]
                score = line.split("\t")[-1].strip()
                item["query_id"] = query_id
                query_used.append(query_id)
                item["doc_id"] = doc_id
                item["score"] = score
                qrel_res.append(item)

    # query file suppose to be the jsonl file 
    with open(query_file,"r") as f:
        lines = f.readlines()
        
        retrieval_json = []
        index = 0
        if data_name == "nfcorpus":
            for line in lines:
                index+=1
                query_item = {}
                query_id = line.split("\t")[0]
                if query_id in query_used:
                    query = line.split("\t")[1].strip()
                    documents = retrieve_documents(query, knowledge_base)
                    documents = find_score(qrel_res, documents, query_id)
                    query_item["hits"] = documents
                    query_item["query"] = query
                    query_item["query_id"] = query_id
                    retrieval_json.append(query_item)
        else:
            for line in lines:
                line = json.loads(line)
                query_item = {}
                query_id = line["_id"]
                if query_id in query_used:
                    query = line["text"]
                    documents = retrieve_documents(query, knowledge_base)
                    documents = find_score(qrel_res, documents, query_id)
            
            
                    query_item["hits"] = documents
                    query_item["query"] = query
                    query_item["query_id"] = query_id
                    retrieval_json.append(query_item)
            
    with open(f"../data/BM25/bm25_{data_name}_{data_type}.json", "w") as f:
        json.dump(retrieval_json, f)

def find_score(qrel_res, documents, query_id):
    for doc in documents:
        docid = doc["docid"]
        flag = False
        for item in qrel_res:
            if item["doc_id"] == docid and item["query_id"] == query_id:
                doc["score"]= item["score"]
                flag=True
        if flag is False:
            doc["score"]= "0"
            
    return documents



In [4]:
for data_name, file_list in dataset_list.items():
    # knowledge_based = data_type
    
    if "knowledge_base" in file_list.keys():
        knowledge_base = file_list["knowledge_base"]
    train_query = file_list["train_query"]
    train_qrel = file_list["train_qrel"]
    
    write_query_qrel(train_query, train_qrel, knowledge_base, data_name ,data_type = "train", train_limit=10000)
    if "test_qrel" in file_list.keys():
        test_qrel = file_list["test_qrel"]
        # If using test, the train_limit must set None
        # write_query_qrel(train_query, test_qrel, knowledge_base,data_name, data_type = "test",  train_limit=None) 
    if "dev_qrel" in file_list.keys():
        dev_qrel = file_list["dev_qrel"]
        write_query_qrel(train_query, dev_qrel, knowledge_base,data_name, data_type = "dev",  train_limit=None) 
        
    

# Prepare reason files

In [1]:
# prepare the reasons from the paper: https://arxiv.org/abs/2304.09542 
import re
from tqdm import tqdm
from rank_gpt_reason import sliding_windows, create_permutation_instruction, run_llm, receive_permutation
import copy


api_key = "sk-proj-9DfFb55gEDX2knzRbykaT3BlbkFJlsVHV2DoNw8pfuSwtdhB"

# gpt-4
def single_search(item=None, rank_start=0, rank_end=100, model_name='gpt-3.5-turbo', api_key=api_key):
  query = item["query"]
  # (1) Create permutation generation instruction
  messages = create_permutation_instruction(item=item, rank_start=rank_start, rank_end=rank_end, model_name=model_name)
  # (2) Get ChatGPT predicted permutation
  permutation = run_llm(messages, api_key=api_key, model_name=model_name)
  # (3) Use permutation to re-rank the passage
  reasoning = permutation
  def clean_reason(permutation):
      # 找到第一次出现的排名序列
      ranking_pattern = re.findall(r'\[\d+\] >', permutation)
      new_response = ""
      # 提取这些模式中的数字
      ranking_numbers = [int(num) for pattern in ranking_pattern for num in re.findall(r'\d+', pattern)]
      new_response = " ".join([str(i )for i in ranking_numbers])
      # 单独处理最后一个排名元素
      # 查找文本中的最后一个 [数字] 模式
      last_number_match = re.search(r'\[\d+\](?!.*\[\d+\])', permutation)
      if last_number_match:
          last_number = last_number_match.group().strip('[]')
          new_response +=  " " + last_number
          # ranking_numbers.append(str(last_number))
      return new_response
  res = clean_reason(permutation)
  response_id = []
  res = res.split()
  # item["hits"] = item["hits"][rank_start: rank_end]
  for i in res:
      response_id.append(item["hits"][int(i)-1]["docid"])
  unsorted_docs = []
  for j in item["hits"][rank_start: rank_end]:
    unsorted_docs.append(j["content"])

  scores = []
  for k in item["hits"][rank_start: rank_end]:
    scores.append(k["score"])
  return_format = {"query":query,
                   "sorted_docids":response_id,
                   "re_rank_id":res,
                   "unsorted_docs": unsorted_docs,
                   "reason": reasoning ,
                   "scores": scores}
  return return_format

# fix with the current sliding window
def file_search(json_file, rank_start=0, rank_end=3,window_size=20, step=10, model_name='gpt-3.5-turbo'):
    return_format = []
    for item in tqdm(json_file):
      res= single_search(item, rank_start=rank_start, rank_end=rank_end, model_name=model_name)
      return_format.append(res)
    return return_format


def sliding_windows(item=None, rank_start=0, rank_end=100, window_size=20, step=10, model_name='gpt-3.5-turbo',
                    api_key=None):
    slide_item = copy.deepcopy(item)
    end_pos = rank_end
    start_pos = rank_end - window_size
    res = []
    while start_pos >= rank_start:
        start_pos = max(start_pos, rank_start)
        
        new_item = single_search(slide_item, start_pos, end_pos, model_name=model_name, api_key=api_key)
        end_pos = end_pos - step
        start_pos = start_pos - step
        res.append(new_item)
    return res
import json



In [5]:
from tqdm import tqdm
import json
import os
#Load BM25 
bm25_list = {
    # "nfcorpus":{
    #     "train":"../data/BM25/bm25_nfcorpus_train.json"}
    # "covid": {
    #     "train":"../data/BM25/bm25_covid_train.json"
    #    },
    # "touche":{
    #     "train":"../data/BM25/bm25_touche_train.json"}
    # "dbpedia":{
    #     "train":"../data/BM25/bm25_dbpedia_train.json",
    #     "test":"../data/BM25/bm25_dbpedia_test.json"},
    # "scifact":{
    #     "train":"../data/BM25/bm25_scifact_train.json",
    #     "test":"../data/BM25/bm25_scifact_test.json"},
    "msmarco":{
        "train":"../data/BM25/bm25_msmarco_train.json",
        "test":"../data/BM25/bm25_msmarco_test.json",
        "dev":"../data/BM25/bm25_msmarco_dev.json"},
    # "noval":{
    #    "train":"../data/BM25/bm25_noval_train.json",
    #     "test":"../data/BM25/bm25_noval_test.json"}     
}


# nf = json.load(open(nf_file))
# touche = json.load(open(touche_file))
# trec_covid = json.load(open(trec_covid_file))


def file_single_search(json_file, rank_start=0, rank_end=5, model_name='gpt-3.5-turbo', data_name="msmarco", data_type = "train"):
    return_format = []
    file_path = f'../data/{model_store}/{data_name}_{data_type}.jsonl'
    if os.path.exists(file_path):
        os.remove(file_path)
    model_store = "GPT3.5" if model_name == 'gpt-3.5-turbo' else "GPT4"
    for item in tqdm(json_file):
      res= sliding_windows(item, rank_start=rank_start, rank_end=rank_end, model_name=model_name)
      return_format.append(res)
      with open(file_path, 'a') as f:
       for res in return_format:
            json.dump(res, f)
            f.write('\n')
    return return_format


def file_sliding_search(json_file, rank_start=0, rank_end=20, window_size=5, step=5, model_name='gpt-3.5-turbo',data_name="msmarco", data_type = "train", index=None):
    return_format = []
    model_store = "GPT3.5" if model_name == 'gpt-3.5-turbo' else "GPT4"
    if index:
        
        file_path = f'../data/{model_store}/{data_name}_{data_type}_{str(index)}.jsonl'
    else:
        file_path = f'../data/{model_store}/{data_name}_{data_type}.jsonl'
    
    # if os.path.exists(file_path):
    #     os.remove(file_path)
    for item in tqdm(json_file):
      res= sliding_windows(item=item, rank_start=rank_start, rank_end=rank_end, window_size=window_size, step=step, model_name=model_name, api_key=api_key)
      # print(res)
      return_format.append(res)
    # print(return_format)
    
    with open(file_path, 'a') as f:
        for res in return_format:
            for r in res:
                json.dump(r, f)
                f.write('\n')
    return return_format
    
for data_name, data in bm25_list.items():
     
    if "test" in data.keys():
        test_data = data["test"]
        test_data = json.load(open(test_data))
        test_data = test_data 
        # Note here there will be 20/5 = 4 for each query 
        file_sliding_search(test_data, rank_start=0, rank_end=20, window_size=5, step=5, model_name='gpt-4',data_name=data_name, data_type="test")

    # if "dev" in data.keys():
    #     dev_data = data["dev"]
    #     dev_data = json.load(open(dev_data))
    #     file_sliding_search(dev_data, rank_start=0, rank_end=20, window_size=5, step=5, model_name='gpt-3.5-turbo',data_name=data_name, data_type="dev")


In [ ]:
# range length 0-9365

for data_name, data in bm25_list.items():
    train_data = data["train"]
    train_data = json.load(open(train_data))
    total_length = len(train_data)
    step2train = 100
    ranges = total_length //step2train
    # print(ranges) # 93
    for idx in range(0, ranges): # right now we have 100, 200, we hope begin at 200 so we begin at 2
        begin_index = idx*step2train
        end_index = (idx+1)*step2train
        print("begin index", begin_index)
        print("end index", end_index)
        temp_train_data = train_data[begin_index:end_index]

        file_sliding_search(temp_train_data, rank_start=0, rank_end=10, window_size=5, step=5, model_name='gpt-4',data_name=data_name, data_type="train", index=end_index)


begin index 0
end index 100


100%|█████████████████████████████████████████████████████████| 100/100 [26:23<00:00, 15.83s/it]


begin index 100
end index 200


100%|███████| 100/100 [27:02<00:00, 16.22s/it]


begin index 200
end index 300


100%|█████████████████████████████████████████████████████████| 100/100 [27:08<00:00, 16.29s/it]


begin index 300
end index 400


 23%|█████████████▎                                            | 23/100 [06:13<21:57, 17.11s/it]